In [15]:
# 필수 라이브러리 호출
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [6]:
# 데이터 정의
st_default_org = pd.read_csv('../CSV/st_default/st_default.csv')
st_default_df = st_default_org.iloc[:, 1:]

In [7]:
# train_test_split 하기 전에 문자형 데이터들 변경 - 1)레이블 vs 2) one-hot
# 그런데 y_label의 경우 default를 예측하는 값이므로 이 값은 label
# 나머지 student는 get_dummies()

# X_data = st_default_df1.drop('default', axis=1)
# y_target = st_default_df1['default']

x_data = st_default_df.iloc[:, 1:]
y_label = st_default_df['default']

# EDA를 통한 데이터 확인
st_default_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   default  10000 non-null  object 
 1   student  10000 non-null  object 
 2   balance  10000 non-null  float64
 3   income   10000 non-null  float64
dtypes: float64(2), object(2)
memory usage: 312.6+ KB


In [10]:
x_data.describe()

,balance,income
count,10000.000000,10000.000000
mean,835.374886,33516.981876
std,483.714985,13336.639563
min,0.000000,771.967729
25%,481.731105,21340.462905
50%,823.636973,34552.644800
75%,1166.308387,43807.729275
max,2654.322576,73554.233500


In [11]:
# 크기 확인 
# print('X_data의 크기:', X_data.shape[0])
# print('y_target의 크기:', y_target.shape[0])

print('X_data의 크기:', x_data.shape[0])
print('y_target의 크기:', y_label.shape[0])

X_data의 크기: 10000
y_target의 크기: 10000


In [14]:
# X_data_df = pd.get_dummies(X_data)

# 문자들을 숫자로 변환한다. - 1) 그냥 문자(학생) 컬럼 없애면 ? O.K. vs
#                             2) 문자 -> one-hot // Label Encoding

x_data_df = pd.get_dummies(x_data) # one-hot :: 문자 -> 레이블 -> 원핫 :: ndarray
x_data_df

,balance,income,student_No,student_Yes
0,729.526495,44361.62507,1,0
1,817.180407,12106.13470,0,1
2,1073.549164,31767.13895,1,0
3,529.250605,35704.49394,1,0
4,785.655883,38463.49588,1,0
...,...,...,...,...
9995,711.555020,52992.37891,1,0
9996,757.962918,19660.72177,1,0
9997,845.411989,58636.15698,1,0
9998,1569.009053,36669.11236,1,0


In [19]:
# y_값에 label encoding

# from sklearn.preprocessing import LabelEncoder

# le_encode = LabelEncoder()
# y_target_scaled = le_encode.fit_transform(y_target)

encode = LabelEncoder()
y_label_scaled = encode.fit_transform(y_label)


# imbalanced data
# pd.Series(y_target_scaled).value_counts()

# y_target값을 확인 = imbalanced_data :: 쏠림 데이터
pd.Series(y_label_scaled).value_counts()

# 333/10000 # 3.33% 부도낸 사람


0    9667
1     333
dtype: int64

In [29]:
# 데이터 분할 - train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_data_df, y_target_scaled, stratify=y_target_scaled)
x_train, x_test, y_train, y_test = train_test_split(x_data_df, y_label_scaled, stratify=y_label_scaled)


In [32]:
# 의사결정나무 적용
dt_clf = DecisionTreeClassifier()
dt_clf.fit(x_train, y_train)

y_pred = dt_clf.predict(x_test)

In [33]:
# 평가지표

print(f'정확도는 :{accuracy_score(y_pred,y_test)}') 

정확도는 :0.9532


#### 배우신 분들을 위한 내용

In [40]:
# 1. 교차검증을 수행
from sklearn.model_selection import cross_val_score

# scores = cross_val_score(dt_clf, x_data_df, y_label_scaled, scoring='accuracy', cv=10)
scores = cross_val_score(dt_clf, x_train, y_train, scoring='accuracy', cv=10)

print(f'교차 검증별 정확도:{np.round(scores, 4)}')
print(f'평균 검증별 정확도:{np.round(np.mean(scores), 4)}')

교차 검증별 정확도:[0.9587 0.9507 0.96   0.944  0.964  0.9533 0.9573 0.9533 0.948  0.9493]
평균 검증별 정확도:0.9539


In [ ]:
# 2. 하이퍼파라미터 튜닝을 수행
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth':[3,4,5,6,7,10],
              'min_samples_split':[2,3,5,10],}
grid_dtree = GridSearchCV(dt_clf, param_grid=parameters, cv=10, refit=True)
grid_dtree.fit(X_train,y_train)

In [ ]:
estimator = grid_dtree.best_estimator_

# pred_gird = grid_dtree.best_estimator_.predict(X_test)
pred_grid = estimator.predict(X_test)
print(f'test datasets accuracy:{accuracy_score(y_test,pred_grid)}')
grid_dtree

In [ ]:
# 1. 왜 데이터의 해당하는 정확도 값이 높게 나왔을까요?
## 데이터의 편향이 높아서?
# 2. 데이터는 어떠한 특징을 지니고 있나요?
## 
# 3. 데이터에는 어떠한 문제점이 있나요?
## 학습을 할 때, 데이터의 편향 때문에 정확한 결과값이 나오지 않음
# 4. 그렇다면 이러한 문제점을 해결하기 위한 방안은 무엇이 있을까요?
# [1] oversampling vs undersampling
# [2] Data augementation - 이미지 분석
# [3] get more data
# [4] change our metrics - precision / recall / f1-score / roc_auc


In [ ]:
# end of file